In [4]:
import numpy as np
from lxml import etree
from pykml.factory import KML_ElementMaker as KML
from pykml.parser import Schema
import pandas as pd

In [7]:
data = pd.read_csv("server_data/site_node_info.csv", sep=",", header=0)
node_id = data["node_id"].as_matrix()
site_name = data["site_id"].as_matrix()
lat = data["lat"].as_matrix()
lon = data["long"].as_matrix()
print node_id
print site_name
print lat
print lon

pm1_loc = str(lon[0]) + "," + str(lat[0])
pm1 = KML.Placemark(
          KML.name(site_name[0] + "_" + str(node_id[0])),
          KML.Point(
            KML.coordinates(pm1_loc)
          )
        )
fld = KML.Folder(pm1)
for i in range(1, len(node_id)):
    pm_loc = str(lon[i]) + "," + str(lat[i])
    pm = KML.Placemark(
             KML.name(site_name[i] + "_" + str(node_id[i])),
             KML.Point(
                KML.coordinates(pm_loc)
        )
    )
    fld.append(pm)
print etree.tostring(fld, pretty_print=True)
outfile = file('AR_sensors.kml','w')
outfile.write(etree.tostring(fld, pretty_print=True))
assert Schema('kml22gx.xsd').validate(fld)

[ 1  2  3  4  5  6  7  8  9 10  1  2  3  4  5  6  7  8  9 10  1  2  3  4  5
  6  7  8  9 10 11  1  2  3  4  5  6  7  8  9 10  1  2  3  4  5  6  7  8  9
 10 11  1  2  3  4  5  6  7  8  9 10  1  2  3  4  5  6  7  8  9 10 11  1  2
  3  4  5  6  7  8  9 10  1  2  3  4  5  6  7  8  9 10 11  1  2  3  4  5  6
  7  8  9 10  1  2  3  4  5  6  7  8  9 10  1  2  3  4  5  6  7  8  9 10  1
  2  3  4  5  6  7  8  9 10]
['Alpha' 'Alpha' 'Alpha' 'Alpha' 'Alpha' 'Alpha' 'Alpha' 'Alpha' 'Alpha'
 'Alpha' 'Bear_Trap' 'Bear_Trap' 'Bear_Trap' 'Bear_Trap' 'Bear_Trap'
 'Bear_Trap' 'Bear_Trap' 'Bear_Trap' 'Bear_Trap' 'Bear_Trap' 'Caples_Lk'
 'Caples_Lk' 'Caples_Lk' 'Caples_Lk' 'Caples_Lk' 'Caples_Lk' 'Caples_Lk'
 'Caples_Lk' 'Caples_Lk' 'Caples_Lk' 'Caples_Lk' 'DollyRice' 'DollyRice'
 'DollyRice' 'DollyRice' 'DollyRice' 'DollyRice' 'DollyRice' 'DollyRice'
 'DollyRice' 'DollyRice' 'Duncan_Pk' 'Duncan_Pk' 'Duncan_Pk' 'Duncan_Pk'
 'Duncan_Pk' 'Duncan_Pk' 'Duncan_Pk' 'Duncan_Pk' 'Duncan_Pk' 'Duncan_Pk'
 'Duncan_Pk